# Les 2 Toepassing Bag of Words
In dit notebook kijken we naar een toepassing van het bag of words model. Eerst wordt er gekeken of de zoektermen van de abstracts ook daadwerkelijk meer voorkomen tussen de 2 verschillende onderzoeksonderwerpen, hiermee wordt een scheiding van de zoektermen verwacht per type abstract.
Daarna wordt vanuit sklearn een naive_bayes classificatie model geimporteerd om te kijken of de verschillende types BoW-encodings impact hebben op de prestatie van een machine learning model.

In [37]:
files = ["resources/cancer_abstract.txt","resources/cancer_2_abstract.txt","resources/fibomalgia_2_abstract.txt","resources/fibromalgia_abstract.txt"]

Voor het maken van de bag of words wordt eerst de tekst samengevoegd, getokeniseerd en daarna uit elkaar gehaald. Hiervoor gebruiken we het argument 2000 max tokens met een minimale frequentie van 2, zodat we een uitgebreide corpus krijgen.

In [58]:
from nlp import file_merger,group_encoder,multi_hot_encoding,frequency_checker,tf_idf_calc
max_tokens = 2000
min_freq = 2
merged_words, len_of_files = file_merger(files)
uncoupled_token_list_of_lists, token_dict = group_encoder(max_tokens,min_freq,merged_words,len_of_files)
print(token_dict)

{1: 'c', 2: 'a', 3: 'n', 4: 'e', 5: 'r', 6: 'l', 7: 's', 8: 'u', 9: 'm', 10: 't', 11: 'y', 12: 'g', 13: 'i', 14: 'o', 15: 'h', 16: 'f', 17: 'b', 18: 'w', 19: 'd', 20: 'v', 21: 'p', 22: ',', 23: '.', 24: 'j', 25: 'k', 26: 'x', 27: 'z', 28: '-', 29: 'q', 30: '(', 31: ')', 32: '1', 33: '8', 34: '9', 35: '0', 36: '%', 37: '—', 38: ';', 39: 'in', 40: 'ti', 41: 'th', 42: 'on', 43: 'an', 44: 'en', 45: 'al', 46: 'er', 47: 'tion', 48: 're', 49: 'the', 50: 'of', 51: 'om', 52: 'es', 53: 'is', 54: 'ed', 55: 'or', 56: 'ic', 57: 'at', 58: 'and', 59: 'ation', 60: 'ar', 61: 'ing', 62: 'ut', 63: 'ent', 64: 'to', 65: 'ac', 66: 've', 67: 'utation', 68: 'og', 69: 'ch', 70: 'as', 71: 'le', 72: 'pr', 73: 'mutation', 74: 'pa', 75: 'un', 76: 'ts', 77: 'ly', 78: 'ec', 79: 'anc', 80: 'dr', 81: 'ver', 82: 'ical', 83: 'su', 84: 'no', 85: 'ol', 86: 'fi', 87: 'ia', 88: 'li', 89: 'mutations', 90: 'id', 91: 'pre', 92: 'mp', 93: 'rom', 94: 'um', 95: 'tic', 96: 'se', 97: 'ig', 98: 'by', 99: 'oc', 100: 'e,', 101: 'fib',

## De zoektermen in tokens  
cancer mutation en fibromalgia  
  
cancer 109  
mutation token 223  
fibromalgia token 395  

Hieronder de generatie van de BoW encodings in een dataframe format.

In [39]:
multi_df = multi_hot_encoding(uncoupled_token_list_of_lists,token_dict,files)
multi_df.head()

,resources/cancer_abstract.txt,resources/cancer_2_abstract.txt,resources/fibomalgia_2_abstract.txt,resources/fibromalgia_abstract.txt
1,1,1,1,1
2,1,1,1,1
3,1,1,1,1
4,1,1,1,1
5,0,0,1,1


In [40]:
freq_df = frequency_checker(uncoupled_token_list_of_lists,token_dict,files," ")
freq_df.head()

,resources/cancer_abstract.txt,resources/cancer_2_abstract.txt,resources/fibomalgia_2_abstract.txt,resources/fibromalgia_abstract.txt
1,3,3,4,2
2,3,3,10,11
3,1,1,4,8
4,3,1,9,9
5,0,0,4,3


In [41]:
tf_df = tf_idf_calc(uncoupled_token_list_of_lists,token_dict,files)
tf_df.head()

,resources/cancer_abstract.txt,resources/cancer_2_abstract.txt,resources/fibomalgia_2_abstract.txt,resources/fibromalgia_abstract.txt
1,0.066980,0.063963,0.036979,0.022647
2,0.055505,0.053005,0.076608,0.103219
3,0.021600,0.020627,0.035774,0.087639
4,0.058403,0.018591,0.072547,0.088862
5,0.000000,0.000000,0.041190,0.037839


## Zoektermen vergelijken met TF-IDF  
Voor het vergelijken van de zoektermen in elk bestand gaan we de TF-IDF waardes vergelijken in het tf_df dataframe  

In [62]:
# token key -1 voor indexing
cancer = 109-1
mutation = 223-1
fibromalgia = 395-1
tf_df.iloc[[cancer,mutation,fibromalgia]]


,resources/cancer_abstract.txt,resources/cancer_2_abstract.txt,resources/fibomalgia_2_abstract.txt,resources/fibromalgia_abstract.txt
109,0.142102,0.067851,0.000000,0.00000
223,0.057732,0.027566,0.000000,0.00000
395,0.000000,0.000000,0.012744,0.01561


Zoals te zien aan het dataframe is er een duidelijk verschil aan TF-IDF waardes tussen de cancer en fibromalgia bestanden

## Machine learning op Bag of Words data
Om te kijken of het type BoW encoding verschill uitmaakt voor het trainen van de data word er op welk van de datasets een GaussianNB model getraind waar alle data van kanker True is een van fibromalgia False. 

In [ ]:
import numpy as np
import pandas as pd

kanker_file_een = tf_df[[files[0]]].to_numpy()
kanker_file_twee = tf_df[[files[1]]].to_numpy()
# data,labels
kanker_files = np.concatenate((kanker_file_een, kanker_file_twee))
kanker_labels = np.ones(len(kanker_files)).astype(bool)

fibro_file_een = tf_df[[files[2]]].to_numpy()
fibro_file_twee = tf_df[[files[3]]].to_numpy()
fibro_files = np.concatenate((fibro_file_een,fibro_file_twee))
fibro_labels = np.zeros(len(fibro_files)).astype(bool)

labels = np.concatenate((kanker_labels,fibro_labels))
data = np.concatenate((kanker_files,fibro_files))

In [64]:
from sklearn import naive_bayes
from sklearn.model_selection import cross_validate
print("GaussianNB op tf-idf encoding")
model = naive_bayes.GaussianNB()
score = cross_validate(model,data,labels, return_train_score=True)
for key, val in score.items():
    print(key , val.mean())


GaussianNB op tf-idf encoding
fit_time 0.004001474380493164
score_time 0.002828073501586914
test_score 0.6003260281385281
train_score 0.6003127346986996


freq diff

In [44]:
kanker_file_een = freq_df[[files[0]]].to_numpy()
kanker_file_twee = freq_df[[files[1]]].to_numpy()
kanker_files = np.concatenate((kanker_file_een, kanker_file_twee))
kanker_labels = np.ones(len(kanker_files)).astype(bool)

fibro_file_een = freq_df[[files[2]]].to_numpy()
fibro_file_twee = freq_df[[files[3]]].to_numpy()
fibro_files = np.concatenate((fibro_file_een,fibro_file_twee))
fibro_labels = np.zeros(len(fibro_files)).astype(bool)

labels = np.concatenate((kanker_labels,fibro_labels))
data = np.concatenate((kanker_files,fibro_files))

In [67]:
model = naive_bayes.GaussianNB()
score = cross_validate(model,data,labels, return_train_score=True)
print("GaussianNB op freqwentie encoding")
for key, val in score.items():
    print(key , val.mean())

GaussianNB op freqwentie encoding
fit_time 0.001813364028930664
score_time 0.0009319305419921875
test_score 0.6003260281385281
train_score 0.6003127346986996


multi def

In [46]:
kanker_file_een = multi_df[[files[0]]].to_numpy()
kanker_file_twee = multi_df[[files[1]]].to_numpy()
kanker_files = np.concatenate((kanker_file_een, kanker_file_twee))
kanker_labels = np.ones(len(kanker_files)).astype(bool)

fibro_file_een = multi_df[[files[2]]].to_numpy()
fibro_file_twee = multi_df[[files[3]]].to_numpy()
fibro_files = np.concatenate((fibro_file_een,fibro_file_twee))
fibro_labels = np.zeros(len(fibro_files)).astype(bool)

labels = np.concatenate((kanker_labels,fibro_labels))
data = np.concatenate((kanker_files,fibro_files))

In [66]:
model = naive_bayes.GaussianNB()
print("GaussianNB op multi-hot encoding")
score = cross_validate(model,data,labels, return_train_score=True)
for key, val in score.items():
    print(key , val.mean())

GaussianNB op multi-hot encoding
fit_time 0.0022960662841796874
score_time 0.0015178680419921874
test_score 0.6003260281385281
train_score 0.6003127346986996


## Vergelijking
Zoals te zijn aan de bovenstaande resultaten zit er geen verschil in test en train scores die de prestatie van het model weergeven. Wel is hier op aan te merken dat de deze scores onder de 0.7 zijn en dus niet voldoen als een praktisch implementeerbaar model. Waar de encodings wel een verschil maken is in de trainingstijd, hierin waren multi-hot en frequentie encoding een stuk sneller dan de tf-idf data. Dit zal waarschijnlijk komen omdat de getallen bij multi-hot en frequentie encoding simpler zijn(int) dan de tf-idf getallen(float).